In [26]:
import numpy as np
import pandas as pd
import yfinance as yf

class Portfolio():
    def __init__(self, cash, stocks):
        self.initial = cash
        self.cash = cash
        self.stocks = stocks
        
    def buy(self, stock, price, x):
        # get stock and price
        stock_buy_amount = np.floor(self.cash / price).astype(int)
        if stock_buy_amount > x:
            stock_buy_amount = x
        stock_price = np.round(stock_buy_amount*price, 2)

        # set money
        self.cash = np.round(self.cash - stock_price, 2)
        if stock in self.stocks:
            self.stocks[stock] += stock_buy_amount
        else:
            self.stocks[stock] = stock_buy_amount
    
    def buy_all(self, stock, price, comission=0):
        # get stock and price
        stock_buy_amount = np.floor(self.cash / price).astype(int)
        stock_price = np.round(stock_buy_amount*price, 2)

        # set money
        self.cash = np.round(self.cash - stock_price, 2)
        if stock in self.stocks:
            self.stocks[stock] += stock_buy_amount
        else:
            self.stocks[stock] = stock_buy_amount
    
    def sell(self):
        pass
    
    def sell_all(self, stock, price):
        sell_price = self.stocks[stock] * price
        comission = (sell_price / 1000) * 0.02
        self.cash = np.round(self.cash + sell_price - comission, 2)
        self.stocks[stock] = 0
    
    def __str__(self):
        return f'${self.cash} // {self.stocks}'

In [53]:
stocks = [
    'AAPL', 'BAC', 'KO', 'AXP', 'WFC', 'KHC', 'JPM', 'USB', 'MCO', 'GS'
]
#stocks = pd.read_csv('../data/s_and_p_500.csv')['Symbol']
drop_list = ['AET', 'APC', 'ANDV']

principle = 5000
total_principle = principle*len(stocks)
total_strat = 0
total_baseline = 0
start = "2018-01-01"
end = "2019-12-31"
years = 5
cadence = 1

# ETF Baseline
baseline_ivv = Portfolio(total_principle, {})
history = yf.Ticker('IVV').history(start=start, end=end).iloc[::cadence, :]
baseline_ivv.buy_all('IVV', history.iloc[0]['Open'])
baseline_ivv.sell_all('IVV', history.iloc[-1]['Open'])
baseline_vug = Portfolio(total_principle, {})
history = yf.Ticker('VUG').history(start=start, end=end).iloc[::cadence, :]
baseline_vug.buy_all('VUG', history.iloc[0]['Open'])
baseline_vug.sell_all('VUG', history.iloc[-1]['Open'])

# Stocks
for stock in stocks:
    # Portfolios
    strat = Portfolio(principle, {stock: 0})
    baseline = Portfolio(principle, {stock: 0})
    trades = 0
    
    # Get history
    history = yf.Ticker(stock).history(start=start, end=end).iloc[::cadence, :]
    
    # Get technical measures
    history['ema_short'] = history['Close'].ewm(span=7, adjust=False).mean()
    history['ema_long'] = history['Close'].ewm(span=20, adjust=False).mean()
    history['ema_7'] = history['Close'].ewm(span=7, adjust=False).mean()
    history['ema_20'] = history['Close'].ewm(span=20, adjust=False).mean()
    history['ema_100'] = history['Close'].ewm(span=100, adjust=False).mean()
    history['bband_std'] = history['Close'].ewm(span=20, adjust=False).std()

    # Generate Baseline
    baseline.buy_all(stock, history.iloc[0]['Open'])
    baseline.sell_all(stock, history.iloc[-1]['Open'])

    # Run Strat
    for i, (date, row) in enumerate(history.iterrows()):
        if row['ema_short'] > row['ema_long']:
            prev_count = strat.stocks[stock]
            strat.buy_all(stock, row['Open'])
            after_count = strat.stocks[stock]
            delta = after_count-prev_count
            if delta != 0:
                trades+=1     
        elif row['ema_long'] > row['ema_short']:
            prev_count = strat.stocks[stock]
            strat.sell_all(stock, row['Open'])
            after_count = strat.stocks[stock]
            delta = after_count-prev_count
            if delta != 0:
                trades+=1
                
        # Sell all at very end to get total
        if i == len(history)-1:
            strat.sell_all(stock, row['Open'])

    print(stock)
    strat_returns_perc = np.round((strat.cash/strat.initial-1)*100, 2)
    baseline_returns_perc = np.round((baseline.cash/baseline.initial-1)*100, 2)
    print(f'Strat Returns:\t\t ${strat.cash-strat.initial:,.2f} // {strat_returns_perc}%')
    print(f'Baseline Returns:\t ${baseline.cash-baseline.initial:,.2f} // {baseline_returns_perc}%')
    print(f'Trades: {trades}')
    print()
    
    total_strat += np.round(strat.cash, 2)
    total_baseline += np.round(baseline.cash, 2)
    

# Print totals
print(f'Stocks In Strategy: {len(stocks)}')
print(f'Total Principle: ${total_principle:,}')

total_strat_returns_perc = np.round((total_strat/total_principle-1)*100, 2)
total_baseline_returns_perc = np.round((total_baseline/total_principle-1)*100, 2)
baseline_ivv_returns_perc = np.round((baseline_ivv.cash/baseline_ivv.initial-1)*100, 2)
baseline_vug_returns_perc = np.round((baseline_vug.cash/baseline_vug.initial-1)*100, 2)

print(f'Strat Returns:\t\t ${total_strat-total_principle:,.2f} // {total_strat_returns_perc}%')
print(f'Baseline Returns:\t ${total_baseline-total_principle:,.2f} // {total_baseline_returns_perc}%')
print(f'IVV S&P500 Returns:\t ${baseline_ivv.cash-baseline_ivv.initial:,.2f} // {baseline_ivv_returns_perc}%')
print(f'VUG Growth Returns:\t ${baseline_vug.cash-baseline_vug.initial:,.2f} // {baseline_vug_returns_perc}%')

AAPL
Strat Returns:		 $4,855.15 // 97.1%
Baseline Returns:	 $3,733.93 // 74.68%
Trades: 17

BAC
Strat Returns:		 $2,052.57 // 41.05%
Baseline Returns:	 $1,132.13 // 22.64%
Trades: 25

KO
Strat Returns:		 $2,123.46 // 42.47%
Baseline Returns:	 $1,413.91 // 28.28%
Trades: 23

AXP
Strat Returns:		 $1,864.10 // 37.28%
Baseline Returns:	 $1,393.19 // 27.86%
Trades: 21

WFC
Strat Returns:		 $634.56 // 12.69%
Baseline Returns:	 $-304.57 // -6.09%
Trades: 21

KHC
Strat Returns:		 $-1,177.15 // -23.54%
Baseline Returns:	 $-2,752.44 // -55.05%
Trades: 25

JPM
Strat Returns:		 $1,798.98 // 35.98%
Baseline Returns:	 $1,776.12 // 35.52%
Trades: 34

USB
Strat Returns:		 $895.69 // 17.91%
Baseline Returns:	 $778.80 // 15.58%
Trades: 23

MCO
Strat Returns:		 $4,514.72 // 90.29%
Baseline Returns:	 $3,126.14 // 62.52%
Trades: 23

GS
Strat Returns:		 $1,177.50 // 23.55%
Baseline Returns:	 $-388.49 // -7.77%
Trades: 25

Stocks In Strategy: 10
Total Principle: $50,000
Strat Returns:		 $18,739.58 // 37.48%
